In [1]:
import os

In [2]:
os.listdir()

['archive', 'Untitled.ipynb', 'grid_0_0.csv', '.ipynb_checkpoints']

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import pymc3 as pm
import pandas as pd
# others as needed

## placeholder for the wall of text

In [5]:
grid = pd.read_csv('grid_0_0.csv')
sample = grid.sample(frac=1) # comment placholder
sample.head() # again

,Unnamed: 0,model_number,star_mass,star_age,star_feh,star_MLT,star_fov,effective_T,luminosity,radius,...,he_core_mass,delta_nu,delta_Pg,nu_max,nu_0_10,nu_0_11,nu_0_12,nu_0_13,nu_0_14,nu_0_15
24877,1063,1064,1.5,0.011926,0.0,1.9,0.018,6971.854544,6.563400,1.758436,...,0.000000,70.923828,0.000000,1364.499814,737.302999,805.757374,874.814384,943.739847,1012.142878,1079.820803
29107,193,194,1.7,0.000001,0.0,1.9,0.018,4063.180221,269.296928,33.162126,...,0.000000,0.921929,0.000000,5.695617,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14441,827,828,1.2,0.003357,0.0,1.9,0.018,4630.761795,1.178177,1.688726,...,0.000000,67.404422,0.000000,1452.266693,756.709337,822.430533,888.113471,953.444916,1018.169001,1082.631181
33784,1470,1471,1.8,1.473395,0.0,1.9,0.018,7368.392144,22.728162,2.929514,...,0.148265,36.130956,2559.927724,573.858388,372.233304,405.940024,439.941288,474.062251,508.198526,542.312406
11466,1257,1253,1.1,5.354230,0.0,1.9,0.018,6148.153737,2.231937,1.318592,...,0.005333,93.533537,0.000000,1895.000552,1067.618879,1160.127416,1252.435973,1345.830552,1440.097034,1533.574175


## Second wall of text

In [6]:
x_cols = ['star_mass', 'star_age']
y_cols = ['luminosity', 'effective_T']
x = np.log10(sample[x_cols].values).astype('float32')
y = np.log10(sample[y_cols].values).astype('float32')

## set up model wall of text

In [11]:
def construct_NN(ann_input, ann_output):
    n_hidden = 2 # here are the layers; although the input and output count as layers, I don't think that's what was meant
    
    init_1 = np.random.randn(x.shape[1], n_hidden).astype('float32')
    init_2 = np.random.randn(n_hidden, n_hidden).astype('float32')
    init_out = np.random.randn(n_hidden).astype('float32')

    with pm.Model() as neural_network:
        # placeholder
        ann_input = pm.Data('ann_input', x)
        ann_output = pm.Data('ann_output', y)
        
        weights_in_1 = pm.Normal('w_in_1', 0, sigma=1,
                                 shape=(x.shape[1], n_hidden),
                                 testval=init_1)
        weights_1_2 = pm.Normal('w_1_2', 0, sigma=1,
                                shape=(n_hidden, n_hidden),
                                testval=init_2)
        weights_2_out = pm.Normal('w_2_out', 0, sigma=1,
                                  shape=(n_hidden,),
                                  testval=init_out)
        
        act_1 = pm.math.tanh(pm.math.dot(ann_input,
                                         weights_in_1))
        act_2 = pm.math.tanh(pm.math.dot(act_1,
                                         weights_1_2))
        act_out = pm.math.sigmoid(pm.math.dot(act_2,
                                              weights_2_out))
        
        out = pm.Normal('out', mu = act_out, observed=ann_output, shape = y.shape)
        
    return neural_network


neural_network = construct_NN(x, y.T)   

ValueError: Input dimension mis-match. (input[0].shape[1] = 2, input[1].shape[1] = 40814)